,# Importar Librerias

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

,# Travelling Salesman Problem (TSP)

El problema del viajero es uno de los problemas más famosos en el campo de la optimizacion computacional. El enunciado clasico es:  

"Sean N ciudades de un territorio. El objetivo es encontrar una ruta que, comenzando y terminando en una ciudad en concreto pase solo una vez por cada una de las ciudades y minimice la distancia recorrida por el viajero."

Es decir, encontrar una permutacion:

$P = \{C_0,C_1,\dots,C_N-1\}$ tal que $d_p = \sum_{i=0}^{N-1} d(c_i,c_{i+1})$ sea minima.

# Ciudades

In [2]:
cities = [
("Mexico City",19.4204503765706, -99.14569125006386),     # Mexico City 
("Quito",-0.22504624887149816, -78.49988938997475),       # Quito
("Miami",25.762008625108713, -80.19106117773524),         # Miami
("San Salvador",13.698816559559196, -89.19139082077601),  # San Salvador
("Mendoza",-32.890254485684615, -68.8454728365394),       # Mendoza
("Guadalajara",20.674513177689914, -103.33814458178117),  # Guadalajara
("Merida",20.969848004507575, -89.59865182312967),        # Merida
("Washington D.C.",38.90971457165538, -77.02266752763703),# Washington D.C.
("Monterrey",25.68095070483323, -100.30007435921115),     # Monterrey
("Managua",12.110415225874924, -86.22824820040704),       # Managua
("Caracas",10.475819042313416, -66.90959192246824),       # Caracas
("Boston",42.35091277211663, -71.05104866254476),         # Boston
("Buenos Aires",-34.609506741669584, -58.39281746089803), # Buenos Aires
("New York",40.71067551341166, -74.01246498137415),       # New York
("Panama City",8.942473452575438, -79.17169679439931),    # Panama City
("Brasilia",-15.664864237549638, -46.621579947655576),    # Brasilia
("Montevideo",-34.90728943056367, -56.18638267328233),    # Montevideo
("Bogota",4.708521026911784, -74.0679815641729)]          # Bogota

df = pd.DataFrame(cities, columns=["city", "lat", "lon"])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattergeo(
    lon=df["lon"],
    lat=df["lat"],
    text=df["city"],
    mode="markers"
))

fig.update_geos(
    visible=False, resolution=110,
    showcountries=True, countrycolor="Black",
    lonaxis=dict(range=[-130, -30]),   # limita la longitud (Oeste hasta Este)
    lataxis=dict(range=[-55, 50])      # limita la latitud (Sur a Norte)
)
# Tamaño Grafica
fig.update_layout(
    width=500, height=500,
    margin=dict(r=0, t=0, l=0, b=0)
)
# Captira de Mapa
fig.show(config={
    "staticPlot": True,       
    "displayModeBar": False
})

In [51]:
# Indice de Ciudades (0,...,17)
n=len(df) 
route = list(range(n))

# Trazar Ruta
lon_route = df.iloc[route]["lon"].tolist()
lat_route = df.iloc[route]["lat"].tolist()
lon_route += [lon_route[0]]
lat_route += [lat_route[0]]

fig.add_trace(go.Scattergeo(
    lon=lon_route,
    lat=lat_route,
    mode="lines",
    line=dict(width=2),
    hoverinfo="skip",
    name="Ruta"
))

fig.show(config={
    "staticPlot": True,       # congela el gráfico (sin pan/zoom/selección)
    "displayModeBar": False
})